# **Modelagem**

## **Variáveis**

$NumCaminhoes_{up,t,tr}$: representa a quantidade de caminhões da transportadora na UP no dia específico.

$DecisaoTransportadoraFazenda_{f,t,tr}$: indica se a transportadora \( tr \) está operando na fazenda \( f \) no dia \( t \) (1 para "sim", 0 para "não").

$DecisaoTransportadoraUP_{up,t,tr}$: indica se a transportadora \( tr \) está transportando a madeira da UP \( up \) no dia \( t \) (1 para "sim", 0 para "não").

$PresencaFazenda_{f}$: indica se a operação está ocorrendo na fazenda \( f \) (1 para "sim", 0 para "não").
.

## **Parâmetros**

$Mapeamento\_UP\_Fazenda$: Mapeamento das Unidades Produtivas (UP) com as fazendas.

$Demanda\_Min\_Diaria_t$: Demanda mínima por dia \( t \).

$Demanda\_Max\_Diaria_t$: Demanda máxima por dia \( t \).

$Tempo\_Ciclo_{tr,up}$: Tempo de ciclo da transportadora \( tr \) para a UP \( up \).

$Frota\_Max\_Transportadora_{tr}$: Frota máxima da transportadora \( tr \).

$Frota\_Min\_Transportadora_{tr}$: Frota mínima da transportadora \( tr \).
.

## **Função Objetivo**
Minimizar a variação diária do BD, ou seja, reduzir as variações diárias na quantidade de madeira (BD) entregue.

$$
\text{Minimizar} \left( \max \left( \sum_{up,tr} v_{up,t,tr} \right) - \min \left( \sum_{up,tr} v_{up,t,tr} \right) \right) \forall t
$$


## **Restrições**

**Restrição de atendimento de demanda mínima e máxima diária**

>$RestricaoDemanda_{t}$: Garante que a quantidade de madeira entregue atende à demanda mínima e máxima da fábrica no dia \( t \).


$$
\text{Demanda}_{\text{min}, t} \leq \sum_{up,tr} v_{up,t,tr} \leq \text{Demanda}_{\text{max}, t} \quad \forall t
$$
.

**Restrição de frota mínima e máxima diária por transportadora**
>$RestricaoFrota_{t,tr}$: Garante que a quantidade de caminhões utilizados pela transportadora \( tr \) está dentro da frota mínima e máxima disponível no dia \( t \).

$$
\text{Frota}_{\text{min}, tr} \leq \sum_{up} NumCaminhoes_{up,t,tr} \leq \text{Frota}_{\text{max}, tr} \quad \forall t, \forall tr
$$
.

**Restrição de volume das Unidades Produtivas**
>$RestricaoVolumeUP_{up,tr}$: Garante que o volume total de madeira transportado da UP \( up \) pela transportadora \( tr \) não excede o volume disponível na UP.

$$
\sum_{t} v_{up,t,tr} \leq \text{Volume}_{up} \quad \forall up, \forall tr
$$
.

**Restrição de qualidade média de RSP**

>$RestricaoMediaRSP_{t}$: Garante que a qualidade média em RSP da madeira entregue está dentro dos limites definidos para o dia \( t \).

$$
\text{RSP}_{\text{min}, t} \times \sum_{up,tr} v_{up,t,tr} \leq \sum_{up,tr} v_{up,t,tr} \times \text{RSP}_{up} \leq \text{RSP}_{\text{max}, t} \times \sum_{up,tr} v_{up,t,tr} \quad \forall t
$$
.

**Restrição de decisão de transporte**
>$RestricaoDecisaoTransporte_{t}$: Garante que cada transportadora \( tr \) opera em uma única fazenda por dia.

$$
\sum_{f} d_{f,t,tr} = 1 \quad \forall t, \forall tr
$$
.

**Restrição de transporte fracionado para UPs menores que 7.000 m³**
> $RestricaoDecisaoTransporteFrac_{t}$: Se uma UP tem menos de 7.000 m³, ela deve ser transportada de uma só vez, sem dividir o transporte em vários dias.

$$
v_{up,t,tr} =
\begin{cases}
   \text{Volume}_{up} & \text{se } \sum_{t,tr} x_{up,t,tr} = 1 \text{ e } \text{Volume}_{up} < 7000 \\
   0 & \text{caso contrário}
\end{cases}
$$
.

**Restrição sobre a troca de fazendas e UPs após completar o volume**
> $DecisaoTransportadoraUP_{up,t,tr}$: Se uma UP começou a ser transportada em um dia \( t \) e não foi totalmente esgotada, ela deve continuar sendo transportada no dia \( t + 1 \).

$$
d_{up,t,tr} \leq d_{up,t+1,tr} \quad \forall t, \forall tr, \forall up
$$
.

**Restrição sobre a presença de transportadoras em fazendas diferentes ao mesmo tempo**
> $RestriçãoPresença_{f,t,tr}$: Cada transportador só pode estar em uma fazenda a cada momento.

$$
\sum_{f} d_{f,t,tr} = 1 \quad \forall t, \forall tr
$$



In [ ]:
# Instalando bibliotecas necessárias
# A biblioteca PuLP é uma biblioteca de otimização linear em Python
# CPLEX é um solver comercial que pode ser usado com PuLP
!pip install pulp
!pip install cplex

# Importando bibliotecas padrão
import pandas as pd  # Para manipulação de dados tabulares
import numpy as np  # Para operações matemáticas
import matplotlib.pyplot as plt  # Para visualização de dados
import sys  # Acessar funções específicas do sistema
import os  # Interface com o sistema operacional
import shutil  # Operações de arquivo de alto nível

# Importando bibliotecas para otimização
from pulp import (
    LpProblem,  # Classe para criar o problema de otimização
    LpContinuous, # Classe para definir variáveis contínuas
    LpVariable,  # Classe para criar variáveis de decisão
    lpSum,  # Função para somar variáveis
    LpMinimize,  # Classe para definir a minimização do problema
    LpInteger,  # Tipo de variável inteira
    LpBinary,  # Tipo de variável binária (0 ou 1)
    LpStatus,  # Classe para verificar o status da solução
    LpConstraint,
    CPLEX_PY  # Solver CPLEX para resolver o problema
)

# Importando bibliotecas para visualização avançada
import seaborn as sns  # Biblioteca de visualização estatística
sns.set(style="darkgrid")  # Configurando o estilo dos gráficos
from mpl_toolkits import mplot3d

# Importando a biblioteca Pyomo, uma alternativa ao PuLP para otimização
! pip install pyomo
import pyomo.environ as pyo

import shutil
print(shutil.which("cbc"))


In [ ]:
%%capture
import sys
import os

if 'google.colab' in sys.modules:
    !pip install idaes-pse --pre
    !idaes get-extensions --to ./bin
    os.environ['PATH'] += ':bin'
import shutil
if not (shutil.which("cbc") or os.path.isfile("cbc")):
    if "google.colab" in sys.modules:
        !apt-get install -y -qq coinor-cbc
    else:
        try:
            !conda install -c conda-forge coincbc
        except:
            pass

assert(shutil.which("cbc") or os.path.isfile("cbc"))

import pyomo.environ as pyo


In [ ]:
# Define o caminho (URL) do arquivo Excel que contém os dados de entrada.
# Este arquivo está hospedado no GitHub e é acessível publicamente.
path = 'https://raw.githubusercontent.com/SabrinaLameiras/TransformacaoDigital/main/Dados/generic_input_case.xlsx'

# Função para ler uma planilha específica do arquivo Excel.
# Esta função ajuda a evitar a repetição de código e torna o código mais limpo.
def read_excel_sheet(sheet_name):
    """
    Lê uma planilha específica de um arquivo Excel.

    Parâmetros:
    - sheet_name (str): Nome da planilha para ler.

    Retorna:
    - DataFrame: Dados da planilha especificada.
    """
    return pd.read_excel(path, sheet_name=sheet_name)

# Lendo cada planilha do arquivo Excel e armazenando em DataFrames correspondentes.

# BD_UP contém informações sobre as Unidades de Produção.
bd_df = read_excel_sheet('BD_UP')

# FROTA contém informações sobre a frota de veículos disponível.
frota_df = read_excel_sheet('FROTA')

# HORIZONTE contém informações sobre o horizonte de planejamento.
horizonte_df = read_excel_sheet('HORIZONTE')

# FABRICA contém informações sobre a fábrica e sua demanda.
fabrica_df = read_excel_sheet('FABRICA')

# ROTA contém informações sobre as rotas de transporte.
rota_df = read_excel_sheet('ROTA')

# GRUA contém informações sobre as gruas disponíveis.
grua_df = read_excel_sheet('GRUA')

In [ ]:
# Criando uma lista de DataFrames e seus respectivos nomes para posterior análise.
dataframes = [bd_df, frota_df, horizonte_df, fabrica_df, rota_df, grua_df]
df_names = ['BD_UP', 'FROTA', 'HORIZONTE', 'FABRICA', 'ROTA', 'GRUA']

def display_info_and_describe(dfs, df_names):
    """
    Exibe informações básicas e estatísticas descritivas de cada DataFrame.

    Parâmetros:
        dfs (list): Lista de DataFrames a serem analisados.
        df_names (list): Lista com os nomes dos DataFrames para etiquetar a saída.

    Retorna:
        None. Imprime as informações diretamente no console.
    """
    for i, df in enumerate(dfs):
        print("\n" + "-" * 40)
        print(f"DataFrame: {df_names[i]}")
        print("-" * 40)

        # Exibe informações básicas do DataFrame, como tipos de dados e contagem de non-null.
        print("\nInfo:")
        print("-" * 5)
        print(df.info())

        # Exibe estatísticas descritivas, como média, desvio padrão, mínimos e máximos.
        print("\nDescribe:")
        print("-" * 9)
        print(df.describe())

        print("\n\n")

# Chama a função para exibir as informações.
display_info_and_describe(dataframes, df_names)

In [ ]:
def detect_anomalies(dfs, df_names):
    """
    Detecta anomalias em vários dataframes e retorna os índices das linhas com anomalias.
    """
    anomalies = {}

    for i, df in enumerate(dfs):
        df_name = df_names[i]
        anomalies[df_name] = {}

        # Detecta valores faltantes
        missing_data = {}
        for col in df.columns:
            missing_rows = df[df[col].isnull()].index.tolist()
            if missing_rows:
                missing_data[col] = {
                    'count': len(missing_rows),
                    'rows': missing_rows
                }
        anomalies[df_name]['missing'] = missing_data

        # Detecta duplicatas
        duplicated_rows = df[df.duplicated()].index.tolist()
        anomalies[df_name]['duplicated'] = {
            'count': len(duplicated_rows),
            'rows': duplicated_rows
        }

        # Detecta outliers usando o método IQR
        anomalies[df_name]['outliers'] = {}
        for col in df.select_dtypes(include=['float64', 'int64']).columns:
            Q1 = df[col].quantile(0.25)
            Q3 = df[col].quantile(0.75)
            IQR = Q3 - Q1
            outlier_rows = df[(df[col] < (Q1 - 1.5 * IQR)) | (df[col] > (Q3 + 1.5 * IQR))].index.tolist()
            if outlier_rows:
                anomalies[df_name]['outliers'][col] = {
                    'count': len(outlier_rows),
                    'rows': outlier_rows
                }

    return anomalies

def display_anomalies(anomalies_result):
    for df_name, anomalies in anomalies_result.items():
        print("\n--------------------------------------")
        print(f"DataFrame: {df_name}")
        print("--------------------------------------")

        # Exibir valores faltantes
        print("\nValores Faltantes:")
        print("-------------------")
        if anomalies['missing']:
            for col, missing_data in anomalies['missing'].items():
                print(f"{col}: {missing_data['count']} valores faltantes nas linhas {missing_data['rows']}")
        else:
            print("Nenhum valor faltante.")

        # Exibir duplicatas
        print("\nDuplicatas:")
        print("------------")
        if anomalies['duplicated']['count']:
            print(f"{anomalies['duplicated']['count']} registros duplicados nas linhas: {anomalies['duplicated']['rows']}")
        else:
            print("Nenhum registro duplicado.")

        # Exibir outliers
        print("\nOutliers:")
        print("----------")
        if anomalies['outliers']:
            for col, outlier_data in anomalies['outliers'].items():
                print(f"{col}: {outlier_data['count']} outliers nas linhas: {outlier_data['rows']}")
        else:
            print("Nenhum outlier detectado.")

        print("\n\n")

# Uso da função de exibição
anomalies_result = detect_anomalies(dataframes, df_names)
display_anomalies(anomalies_result)



In [ ]:
def exibir_valores_unicos(*dataframes):
    for df in dataframes:
        print("="*50)
        print(f"DataFrame: {df.name if 'name' in dir(df) else 'Unnamed'}")  # tenta exibir o nome do DataFrame, se disponível
        print("="*50)
        for col in df.columns:
            print(f"Coluna: {col}")
            print(df[col].unique())
            print("-"*50)

# identificação na saída
bd_df.name = "BD_UP"
frota_df.name = "FROTA_DF_DF"
horizonte_df.name = "HORIZONTE_DF"
fabrica_df.name = "FABRICA_DF"
rota_df.name = "ROTA_DF"
grua_df.name = "GRUA_DF"

# Usando a função
exibir_valores_unicos(bd_df, frota_df, horizonte_df, fabrica_df, rota_df, grua_df)

In [ ]:
horizonte_df = horizonte_df.drop(columns='CICLO_LENTO')
frota_df = frota_df.drop(columns='DIA')
bd_df = bd_df.drop(columns=['DATA_COLHEITA','IDADE_FLORESTA', 'IMA', 'RD', 'CLONE', 'ESPECIE', 'PRECIPITACAO', 'Unnamed: 13'])


In [ ]:
# Mapeamento das Unidades Produtivas (UP) com as fazendas
Mapeamento_UP_Fazenda = tuple(zip(bd_df.UP, bd_df.FAZENDA))

# Demanda mínima por dia
Demanda_Min_Diaria = dict(zip(fabrica_df.DIA, fabrica_df.DEMANDA_MIN))

# Demanda máxima por dia
Demanda_Max_Diaria = dict(zip(fabrica_df.DIA, fabrica_df.DEMANDA_MAX))

# Tempo de ciclo de cada transportadora e sua origem
Tempo_Ciclo = dict(zip(tuple(zip(rota_df.TRANSPORTADOR, rota_df.ORIGEM)), rota_df.TEMPO_CICLO))

# Frota máxima de cada transportadora
Frota_Max_Transportadora = dict(zip(frota_df.TRANSPORTADOR, frota_df.FROTA_MAX))

# Frota mínima de cada transportadora
Frota_Min_Transportadora = dict(zip(frota_df.TRANSPORTADOR, frota_df.FROTA_MIN))


In [ ]:
# create a model
model = pyo.ConcreteModel()

In [ ]:
# Conjuntos
model.ConjuntoUPFazenda = pyo.Set(initialize=Mapeamento_UP_Fazenda, dimen=2)
model.Fazenda = pyo.Set(initialize = list(set([fazenda for (_, fazenda) in model.ConjuntoUPFazenda])))
model.TransportadoraParaUP = pyo.Set(initialize=Tempo_Ciclo.keys(), dimen=2)
model.Dias = pyo.Set(initialize=list(range(1, 32)), doc='Dias do mês')
model.Transportadora = pyo.Set(initialize=list(set([transportadora for (transportadora, _) in model.TransportadoraParaUP])), doc='Transportadoras')
model.UP = pyo.Set(initialize=list(set([up for (_, up) in model.TransportadoraParaUP])), doc='Unidades Produtivas')


In [ ]:
# Parâmetros
model.DemandaMin = pyo.Param(model.Dias, initialize=Demanda_Min_Diaria, doc='Demanda mínima no dia')
model.DemandaMax = pyo.Param(model.Dias, initialize=Demanda_Max_Diaria, doc='Demanda máxima no dia')
model.CargaCaminhao = pyo.Param(initialize=66, doc = 'Capacidade de carga de cada caminhão')
model.RSPMinimo = pyo.Param(model.Dias, initialize=dict(zip(fabrica_df.DIA, fabrica_df.RSP_MIN)), doc='RSP mínimo por dia')
model.RSPMaximo = pyo.Param(model.Dias, initialize=dict(zip(fabrica_df.DIA, fabrica_df.RSP_MAX)), doc='RSP máximo por dia')
model.QualidadeUP = pyo.Param(model.UP, initialize=dict(zip(bd_df.UP, bd_df.RSP)), doc='Qualidade em RSP de cada UP')
model.CicloTransportadora = pyo.Param(model.Transportadora, model.UP, initialize=Tempo_Ciclo, doc='Tempo de ciclo da transportadora para a UP')
model.FrotaMinima = pyo.Param(model.Transportadora, initialize=Frota_Min_Transportadora, doc='Frota mínima da transportadora')
model.FrotaMaxima = pyo.Param(model.Transportadora, initialize=Frota_Max_Transportadora, doc='Frota máxima da transportadora')
model.VolumeUP = pyo.Param(model.UP, initialize=dict(zip(bd_df.UP, bd_df.VOLUME)), doc='Volume disponível na UP')
model.RSPUP = pyo.Param(model.UP, initialize=dict(zip(bd_df.UP, bd_df.DB)), doc='RSP em cada UP')


In [ ]:
# Construção do conjunto TransportadoraFazendaUP
RelacaoTransportadoraFazendaUP = []
for (up, fazenda) in model.ConjuntoUPFazenda:
    for (transportadora, up_origem) in model.TransportadoraParaUP:
        if up == up_origem:
            RelacaoTransportadoraFazendaUP.append(list([transportadora, fazenda, up]))
model.ConjuntoTransportadoraFazendaUP = pyo.Set(initialize = RelacaoTransportadoraFazendaUP)


In [ ]:
# Variáveis
model.NumCaminhoes = pyo.Var(model.ConjuntoTransportadoraFazendaUP, model.Dias, domain=pyo.NonNegativeReals, doc='Quantidade de caminhões da transportadora na UP no dia específico')
model.DecisaoTransportadoraFazenda = pyo.Var(model.ConjuntoTransportadoraFazendaUP, model.Dias, domain=pyo.Boolean, doc='Decisão da transportadora de estar na fazenda em um dia específico')
model.DecisaoTransportadoraUP = pyo.Var(model.TransportadoraParaUP, model.Dias, domain=pyo.Boolean, doc='Decisão da transportadora de estar na UP em um dia específico')
model.PresencaFazenda = pyo.Var(model.Fazenda, domain=pyo.Boolean, doc='Decisão de operar na fazenda específica')


In [ ]:
# Restrição de atendimento de demanda mínima e máxima diária
model.RestricaoDemanda = pyo.ConstraintList()
for dia in model.Dias:
    centro = sum(model.NumCaminhoes[transportadora, fazenda, up, dia] * model.CicloTransportadora[transportadora, up] * model.CargaCaminhao for (transportadora, fazenda, up) in model.ConjuntoTransportadoraFazendaUP)
    minimo = model.DemandaMin[dia]
    maximo = model.DemandaMax[dia]
    model.RestricaoDemanda.add(centro >= minimo)
    model.RestricaoDemanda.add(centro <= maximo)

In [ ]:
# Restrição de frota mínima e máxima diária por transportadora
model.RestricaoFrota = pyo.ConstraintList()
for dia in model.Dias:
    for transportadora in model.Transportadora:
        centro = sum(model.NumCaminhoes[transportadora, :, :, dia])
        minimo = model.FrotaMinima[transportadora]
        maximo = model.FrotaMaxima[transportadora]
        model.RestricaoFrota.add(centro >= minimo)
        model.RestricaoFrota.add(centro <= maximo)

In [ ]:
# Restrição de volume das Unidades Produtivas
model.RestricaoVolumeUP = pyo.ConstraintList()
for (transportadora, fazenda, up) in model.ConjuntoTransportadoraFazendaUP:
    centro = sum(model.NumCaminhoes[transportadora, fazenda, up, dia] * model.CicloTransportadora[transportadora, up] * model.CargaCaminhao for dia in model.Dias)
    maximo = model.VolumeUP[up]
    model.RestricaoVolumeUP.add(centro <= maximo)

In [ ]:
# Restrição de qualidade média de RSP
model.RestricaoMediaRSP = pyo.ConstraintList()
for dia in model.Dias:
    numerador = sum(model.NumCaminhoes[transportadora, fazenda, up, dia] * model.CicloTransportadora[transportadora, up] * model.CargaCaminhao * model.QualidadeUP[up] for (transportadora, fazenda, up) in model.ConjuntoTransportadoraFazendaUP)
    denominador = sum(model.NumCaminhoes[transportadora, fazenda, up, dia] * model.CicloTransportadora[transportadora, up] * model.CargaCaminhao for (transportadora, fazenda, up) in model.ConjuntoTransportadoraFazendaUP)
    minimo = model.RSPMinimo[dia]
    maximo = model.RSPMaximo[dia]
    model.RestricaoMediaRSP.add(numerador >= minimo * denominador)
    model.RestricaoMediaRSP.add(numerador <= maximo * denominador)


In [ ]:
# Restrição de decisão de transporte
model.RestricaoDecisaoTransporte = pyo.ConstraintList()
for dia in model.Dias:
    for (transportadora, fazenda, up) in model.ConjuntoTransportadoraFazendaUP:
        centro = sum(model.DecisaoTransportadoraFazenda[transportadora, :, :, dia])
        model.RestricaoDecisaoTransporte.add(centro == 1)



In [ ]:
def FuncaoObjetivo(model):
    return sum(model.NumCaminhoes[transportadora, fazenda, up, dia] * model.RSPUP[up] for (transportadora, fazenda, up) in model.ConjuntoTransportadoraFazendaUP for dia in model.Dias)

model.FuncaoObjetivo = pyo.Objective(sense=pyo.minimize, rule=FuncaoObjetivo)


In [ ]:
import shutil
print(shutil.which("cbc"))

resultado = pyo.SolverFactory('cbc').solve(model)
resultado.write()

In [ ]:
resultados = []

# Cálculos que não dependem de transportadora ou fazenda
RSP_total = sum(pyo.value(model.NumCaminhoes[t, f, u, dia]) * model.CicloTransportadora[t, u] * model.CargaCaminhao * model.QualidadeUP[u] for dia in model.Dias for (t, f, u) in model.ConjuntoTransportadoraFazendaUP)
transporte_diario = sum(pyo.value(model.NumCaminhoes[t, f, u, dia]) * model.CicloTransportadora[t, u] * model.CargaCaminhao for dia in model.Dias for (t, f, u) in model.ConjuntoTransportadoraFazendaUP)

for dia in model.Dias:
    for (transportadora, fazenda, UP) in model.ConjuntoTransportadoraFazendaUP:
        qtd = pyo.value(model.NumCaminhoes[transportadora, fazenda, UP, dia])

        # Verificar se a variável foi inicializada e é diferente de zero
        if qtd is not None and qtd != 0:
            volume = qtd * model.CicloTransportadora[transportadora, UP] * model.CargaCaminhao
            resultados.append([UP, fazenda, transportadora, dia, pyo.value(model.QualidadeUP[UP]), RSP_total/transporte_diario, qtd, volume])

# Criando um DataFrame com os resultados
df_resultados = pd.DataFrame(resultados, columns=['UP', 'Fazenda', 'Transportadora', 'Dia', 'DB', 'RSP', 'Quantidade', 'Volume'])
import matplotlib.pyplot as plt
import seaborn as sns

def plot_green_colors_bar_chart(df_resultados):
    # Ordenar o dataframe pela quantidade
    df_sorted = df_resultados.groupby('Fazenda').sum()['Quantidade'].sort_values(ascending=True).reset_index()

    # Calcular o tamanho da figura com base no número de fazendas
    fig_height = max(6, len(df_sorted) * 0.6)

    # Inicializar layout com tamanho calculado
    fig, ax = plt.subplots(figsize=(6, fig_height))

    # Adicionar fundo branco ao ax e fig
    ax.set_facecolor('white')
    fig.set_facecolor('white')

    # Escolher cores com base na cor verde
    bar_colors = sns.color_palette("Greens", len(df_sorted))

    # Criar o gráfico com altura ajustada das barras
    bar_height = 0.5
    ax.barh(df_sorted['Fazenda'], df_sorted['Quantidade'], color=bar_colors, zorder=2, height=bar_height)

    # Adicionar linhas verticais cinza
    ax.grid(linestyle='-', alpha=0.5, color='lightgray', axis='x')

    # Título do gráfico com espaçamento e tamanho de fonte ajustados
    title = 'Distribuição da Quantidade de DB por Fazenda'
    fig.text(0.03, 1.02, title, fontsize=18, fontweight='bold', ha='left', color='black', family='dejavu sans')

    # Subtítulo do gráfico com espaçamento e tamanho de fonte ajustados
    subtitle = 'Quantidade acumulada por Fazenda'
    fig.text(0.03, 0.98, subtitle, fontsize=10, color='black', ha='left', family='dejavu sans')

    # Remover as bordas do gráfico
    for spine in ax.spines.values():
        spine.set_visible(False)

    # Configurar posição e etiquetas dos eixos com tamanho de fonte ajustado
    ax.tick_params(axis='y', labelsize=9, colors='black')
    ax.tick_params(axis='x', colors='black')
    ax.xaxis.tick_top()

    # Ajustar margens
    plt.subplots_adjust(left=0.2, right=0.8, top=0.9, bottom=0.1)

    # Exibir o gráfico
    plt.show()

plot_green_colors_bar_chart(df_resultados)

# Save df_resultados to a CSV file
df_resultados.to_csv('resultados.csv', index=False)

